# 2020-03-23 Workshop: Localization Understanding RF Signal

The simplest indoor localization is RSS fingerprinting using alpha-trimmed filters and kNN. It is an easier but less accurate method than using Bayesian or particle filters. To understand why that is, we first discuss RF signal attenuation indoors.


## RF Signal Attenuation

Attenuation refers to any reduction in the strength of a signal. It is a natural consequence of signal transmission over long distances and through challenging medien. Attenuation is represented in decibels (dB), which is ten times the logarithm of the signal power at a particular input divided by the signal power at an output of a specified medium. 

<div class="alert alert-block alert-info">
<b>Example:</b> An office wall (the specific medium) that changes the propagation of an RF signal from a power level of 10 milliwatts (the input) to 5 milliwatts (the output) represents 3 dB of attenuation. Consequently, positive attenuation causes signals to become weaker when travelling through the medium.
</div>

Longer distance and higher frequency lead to more attenuation. Localization with perfect RF signals is trivial, but there are no perfect signals in the wild!

<img src="img/ws2/rf_propagation_effects.png" width="600">

### Why does signal attenuation vary in a static environment?

It's the atmosphere, humidity, density, air-currents particles etc. that subtly change the propagation. The presence of the shadowing/objects magnifies the effect.

<img src="img/ws2/real_rf_signal.png" width="600">


### 1. Scattering

Scattering occurs when a wave impinges upon an object with dimensions on the order of l or less, causing the reflected energy to spread out or "scatter" in many directions. Small objects such as street lights, signs, & leaves cause scattering.

### 2. Reflection

* Occurs when a wave impinges upon a smooth surface.
* Dimensions of the surface are large relative to l.
* Reflections occur from the surface of the earth and from buildings and walls.

### 3. Diffraction (Shadowing)

* Occurs when the path is blocked by an object with large dimensions relative to l and sharp irregularities (edges).
* Secondary “wavelets” propagate into the shadowed region.
* Diffraction gives rise to bending of waves around the obstacle.

### 4. Absorption (Large-scale fading)

You can't do much against large scale fading: tearing down a wall is not a daily option. In a way, large scale fading is good:
* it helps in characterizing specific spaces,
* localization in the corridor is harder than in rooms

There is one absorption effect that causes problems, but that can be overcome to some extent: 
body effects. The human body is both a reflector and an absorber of RF energy. This is likely to be even more apparent at higher frequencies, e.g. 2.4 GHz. The human body is made up primarily of "salt water". Salt water is an effective absorber of RF energy.  (Submarines have to surface to send FM radio signals). The more body fat a person has, the more RF is absorbed.

### 5. Multipath fading (Small-scale fading)

Due to propagation mechanisms, multiple waves arrive at the receiver. Multipath propagation causes large and rapid fluctuations in a signal. These fluctuations are not the same as the propagation path loss.

Multipath causes three major problems:
* Rapid changes in signal strength over a short distance or time.
* Random frequency modulation due to Doppler Shifts on different multipath signals.
* Time dispersion caused by multipath delays

The relationship between the size of object and the length of the wavelength is key: needle -- negligible multipath effect, big rock -- major multipath effect.

<img src="img/ws2/multipath.png" width="600">

Propagation along multiple paths leads to self interference. Minor changes in environment can lead to dramatic changes in signal strength. Test small scale fading with your phone. Measure the RSS signal at day and night.

<div class="alert alert-block alert-info">
<b>Hint:</b> Measure n RSS samples: 1 sample per second for 2 minutes should be ok.
</div>


## Alpha-Trimmed Filters

RSS may vary considerably over time. Since mean is highly sensitive to outliers, there are better methods to estimate average expected signal strength, e.g., alpha-trimmed filters. Alpha-trimmed mean filter is windowed filter of nonlinear class, by its nature is hybrid of the mean and median filters. The basic idea behind filter is for any element of the signal (image) look at its neighborhood, discard the most atypical elements and calculate mean value using the rest of them. Alpha is the parameter responsible for the number of trimmed elements.

<img src="http://www.librow.com/content/common/images/articles/article-7/fig-1.gif">

Source: <a href="http://www.librow.com/articles/article-7">Alpha-trimmed mean filter</a>


## RSS Fingerprinting with kNN

<div class="alert alert-block alert-success">
<b>Read</b> (because you are sort of recreating this work): <a href="http://www.cs.columbia.edu/~drexel/CandExam/RADAR_in_building_RF.pdf">RADAR: an in-building RF-based user location and tracking system</a>, IEEE Infocom 2000
</div>

<img src="img/ws2/rf_fingerprinting.png" width="600">

Try this at home first. Recall RF signals are naughty -- use alpha-trimmed filters! This fingerprinting method is still very susceptible to changes in environment.


## Code Snippets

### Initialize WiFi

```Java
public class WifiActivity extends Activity {
	WifiManager wifi;       
		
	@Override
	protected void onCreate(Bundle savedInstanceState) {
        ...
        // Initialize WifiManager (contains API for managing all aspects of Wi-Fi connectivity)
        wifi = (WifiManager) getSystemService(Context.WIFI_SERVICE);
        // If Wifi is switched off, enable it
        if (wifi.isWifiEnabled() == false) 
            wifi.setWifiEnabled(true);
        
        // Register a receiver where scan results are made available
        registerReceiver (new WifiReceiver(), new IntentFilter(WifiManager.SCAN_RESULTS_AVAILABLE_ACTION));
        // begin scan
        wifi.startScan();
	}
...
```

### Get RSSI

```Java
    // WifiReceiver class is subclass of WifiActivity
    class WifiReceiver extends BroadcastReceiver {
        // An access point scan has completed and results are sent here    
        public void onReceive(Context c, Intent intent) {
            //  Call getScanResults() to obtain the results    
            List<ScanResults> results = wifi.getScanResults();
                  
            try {
                for (int n = 0; n < results.size (); n++) {
                    // SSID contains name of AP and level contains RSSI
                    Log.i("Wifi", "SSID = " + results.get(n).SSID + “; RSSI =  “ + results.get(n).level);
                }
            }
            catch (Exception e) {  }
        }
    } // End of class WifiReceiver
} // End of class WifiActivity

```

### Add Permissions in AndroidManifest.xml

```
    <uses-permission android:name="android.permission.ACCESS_COARSE_LOCATION"/>
    <uses-permission android:name="android.permission.ACCESS_FINE_LOCATION"/>
    <uses-permission android:name="android.permission.ACCESS_WIFI_STATE" />
    <uses-permission android:name="android.permission.CHANGE_WIFI_STATE" />
    <uses-permission android:name="android.permission.ACCESS_NETWORK_STATE" />
```

### Enable Your App Access Location Services

```
Settings -> Apps -> YOUR_APP_NAME -> Permissions -> Location = On
```

***
# Credits
* Marco Zuniga's <a href="http://studiegids.tudelft.nl/a101_displayCourse.do?course_id=40368">"Smart Phone Sensing" Course at TU Delft</a>
